# Data Exploration

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_full = pd.read_excel('cordata.xlsx') # contains all data
df_Al = pd.read_excel('cordata_Al.xlsx') # only Al alloy class

FileNotFoundError: [Errno 2] No such file or directory: 'cordata.xlsx'

In [3]:
print(df_full.info())

NameError: name 'df_full' is not defined

In [4]:
print(df_Al.info())

NameError: name 'df_Al' is not defined

In [5]:
# number of entries for every alloy class in the full dataset
for metal in df_full['Metal'].unique():
    print(f"{metal}: {len(df_full[df_full['Metal'] == metal])}")

NameError: name 'df_full' is not defined

### since Al has the biggest number it is indeed smart to start the alloy specific analysis with Al

In [6]:
# unique compounds for full dataset
df_full.groupby('SMILES').agg({'Efficiency':['median', 'std']})

NameError: name 'df_full' is not defined

In [7]:
# unique compounds for Al only dataset
df_Al.groupby('SMILES').agg({'Efficiency':['median', 'std']})

NameError: name 'df_Al' is not defined

### 402 compounds for full, 177 for only Al. How about if we work for the largest alloy group?

In [8]:
def value_counts(df, value):
    # Count unique values in the 'value' column
    value_counts = df[value].value_counts()
    ranked_values = value_counts.sort_values(ascending=False)
    print("Rank of unique values:")
    print(ranked_values)

value_counts(df_full, 'Alloy')
print()
print(f'Unique alloys in full dataset: {len(df_full["Alloy"].unique())}')
print(f'Unique alloys in only Al dataset: {len(df_Al["Alloy"].unique())}')

NameError: name 'df_full' is not defined

### I suggest we start from AA2024, as mild steel is too general of a category.

In [9]:
# composition of AA2000 series is nearly identical, grab alloys from this series
df_AA2024 = df_Al[df_Al['Alloy'].isin(['AA2024', 'AA2014', 'AA2017A'])]
df_AA2024.groupby('SMILES').agg({'Efficiency':['median', 'std']})

NameError: name 'df_Al' is not defined

### 177 compounds for all Al, to 123 compounds to only AA2024.

In [10]:
# you can play around with the column names to see what unique values are in each column
column_name = 'Time_h'
print(df_Al[column_name].unique())
print((len(df_Al[column_name].unique())))

NameError: name 'df_Al' is not defined

In [11]:
# previous counter fuinction to check distribution
value_counts(df_Al, 'Time_h')

NameError: name 'df_Al' is not defined

In [12]:
# you can play around with histograms below to check distribution
# 
# # Plot histogram
import matplotlib.pyplot as plt

def plot_hist(df, value, name):
    plt.hist(df[value], bins=50, color='skyblue', edgecolor='black')
    plt.xlabel(f'{value}')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of {name}')
    plt.grid(True)
    plt.show()

hist_value = 'Time_h'
plot_hist(df_full, value = hist_value, name = 'Full')
plot_hist(df_Al, value = hist_value, name = 'Al')
plot_hist(df_AA2024, value = hist_value, name = 'AA2024')

NameError: name 'df_full' is not defined

In [ ]:
# check filtering to see the effect of getting rid of some rows, to check whether its feasible to drop some columns

def check_filter(dataset, column_name, filter):
    print(f"Number of entries in full dataset: {len(dataset)}")
    print(f"Number of entries in full dataset with {column_name} >= {filter}: {len(dataset[dataset[column_name] >= filter])}")
    print(f"Number of entries in full dataset with {column_name} < {filter}: {len(dataset[dataset[column_name] < filter])}")


check_filter(dataset = df_Al, column_name = 'Time_h',  filter = 1)



Number of entries in full dataset: 2011
Number of entries in full dataset with Time_h >= 1: 1638
Number of entries in full dataset with Time_h < 1: 373


In [ ]:
# quick check to check the amount of unique compounds
filtered_df_Al = df_AA2024[df_AA2024['Time_h'] >= 1] # 123 to 121 compounds, maybe not much information loss for the small dataset...
len(filtered_df_Al['SMILES'].unique())

121

### Now to create a new dataframe cleaned of unnecessary details for analysis. 
Entries with synergy and encapsulation is removed, then unnecessary columns dropped.

In [ ]:
def filter_dataframe_AA2024(df):
    no_synergy_df = df[df['Synergistic_inhib'] == 'No']
    no_encapsulation_df = no_synergy_df[no_synergy_df['Encapsulated'] == 'No']
    filtered_df = no_encapsulation_df.drop(columns=['Inhibitor', 'Number', 'Metal', 'Alloy', 'Temperature_K', 'Salt_Concentrat_M', 
                                                    'Synergistic_inhib','Synergistic_inhib_type', 'Synergistic_inhib_Concentrat_M',
                                                    'Encapsulated', 'Methodology','Reference', 'Link', 'Contributor'])
    return filtered_df

filtered_df_AA2024 = filter_dataframe_AA2024(df_AA2024)
len(filtered_df_AA2024['SMILES'])


611

### 611 datapoints for actual Bayesian optimization work. 

In [ ]:
# save
filtered_df_AA2024.to_excel('filtered_AA2024.xlsx', index=False)

In [ ]:
def filter_dataframe_Al(df):
    no_synergy_df = df[df['Synergistic_inhib'] == 'No']
    no_encapsulation_df = no_synergy_df[no_synergy_df['Encapsulated'] == 'No']
    filtered_df = no_encapsulation_df.drop(columns=['Inhibitor', 'Number', 'Metal', 'Temperature_K', 'Salt_Concentrat_M', 
                                                    'Synergistic_inhib','Synergistic_inhib_type', 'Synergistic_inhib_Concentrat_M',
                                                    'Encapsulated', 'Methodology','Reference', 'Link', 'Contributor'])
    return filtered_df

filtered_df_Al = filter_dataframe_Al(df_Al)
len(filtered_df_Al['SMILES'])

1966

### 1966 datapoints, almost 3 times. Assuming we can just ignore the effect of alloy type, or find a way to featurize it, would be fun to work with. Maybe composition based?

In [ ]:
# save
filtered_df_Al.to_excel('filtered_Al.xlsx', index=False)

In [13]:
def filter_dataframe_full(df):
    no_synergy_df = df[df['Synergistic_Inhib_type'].isnull()]
    filtered_df = no_synergy_df.drop(columns=['Inhibitor', 'Index', 'Mol._weight', 'Temperature_K', 'Salt_Concentrat_M',
                                              'Synergistic_Inhib_type', 'Synergistic_Inhib_M','Methodology','Reference',
                                              'Contributor'])
    return filtered_df

filtered_df_full = filter_dataframe_full(df_full)
len(filtered_df_full['SMILES'])

NameError: name 'df_full' is not defined

### 4708 datapoints, we more than double the previous. Again if we can jump to the featurization of alloy or substrate, super cool number to work with.

In [14]:
# save
filtered_df_full.to_excel('filtered_full.xlsx', index=False)

NameError: name 'filtered_df_full' is not defined